In [1]:
import json
import os
import mp
import numpy as np
from collections import defaultdict
from glob import glob
from tqdm import tqdm
import soundfile as sf
import re

timestamps = [i * 0.02 for i in range(1500 + 1)]

In [2]:
from huggingface_hub import hf_hub_download
import fasttext

filename = hf_hub_download(
    repo_id="mesolitica/fasttext-language-detection-bahasa-en", 
    filename="fasttext.ftz"
)
lang_model = fasttext.load_model(filename)

In [3]:
def chunk(alignment, reject = -8, minimum_length = 1.0):
    alls, temp = [], []
    for a in alignment:
        if a['score'] <= reject:
            if len(temp):
                temp[-1]['end'] = a['start']
                if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
                    alls.append(temp)
                temp = []
        else:
            temp.append(a)
            
    if len(temp):
        if (temp[-1]['end'] - temp[0]['start']) >= minimum_length:
            alls.append(temp)
    return alls

In [4]:
data = defaultdict(list)
with open('prepared-pseudolabel-malaya.jsonl') as fopen:
    for no, l in tqdm(enumerate(fopen)):
        l = json.loads(l)
        data[l['audio_filename']].append((no, l))
        
len(data)

1089630it [00:03, 309539.74it/s]


882371

In [5]:
rows = list(data.values())
len(rows)

882371

In [7]:
# !rm -rf prepared-pseudolabel-malaya-chunks
!mkdir prepared-pseudolabel-malaya-chunks

mkdir: cannot create directory ‘prepared-pseudolabel-malaya-chunks’: File exists


In [8]:
def loop(data):
    data, _ = data
    new_data = []
    for d in tqdm(data):
        
        aligns, scores = [], []
        for i in d:
            f = f'prepared-pseudolabel-malaya_alignment/{i[0]}.alignment'
            try:
                with open(f) as fopen:
                    align = json.load(fopen)
                    score = np.sum([s['score'] for s in align])
                    aligns.append(align)
                    scores.append(score)
            except:
                aligns.append([])
                scores.append(-9999)

        argmax = np.argmax(scores)
        no = d[argmax][0]
        text = d[argmax][1]['new_text']
        lang = text.split('<|startoftranscript|><|')[1].split('|')[0]
        cleaned_text = re.sub(r"<\|.*?\|>", "", text).strip()
        if lang_model.predict(cleaned_text)[0][0] == '__label__english':
            predict_lang = 'en'
        else:
            predict_lang = 'ms'
        
        chunks = chunk(aligns[argmax])
        audio_filename = d[argmax][1]['audio_filename']
        if len(chunks):
            y, sr = sf.read(audio_filename)
            for k, c in enumerate(chunks):
                
                skip = False
                for c_ in c:
                    if (c_['end'] - c_['start']) > 2:
                        skip = True
                        break
                if skip:
                    continue
                    
                new_f = os.path.join('prepared-pseudolabel-malaya-chunks', f'{no}-{k}.mp3')
                if not os.path.exists(new_f):
                    sf.write(new_f, y[int(sr * c[0]['start']): int(sr * c[-1]['end'])], sr)
                ts = []
                
                min_t = min([c_['start'] for c_ in c])
                
                for c_ in c:
                    start = min(timestamps, key=lambda t: abs(t - (c_['start'] - min_t)))
                    end = min(timestamps, key=lambda t: abs(t - (c_['end'] - min_t)))
                    w = c_['text']
                    t = f"<|{start:.2f}|> {w}<|{end:.2f}|>"
                    ts.append(t)
                ts = ''.join(ts)
                new_text = text = f"<|startoftranscript|><|{predict_lang}|><|transcribeprecise|>{ts}<|endoftext|>"
                new_data.append({
                    'audio_filename': new_f,
                    'new_text': new_text,
                })
                
    return new_data

In [9]:
# r = loop((rows[:100], 0))
# len(r)

In [10]:
r = mp.multiprocessing(rows, loop, cores = 10)

100%|██████████████████████████████████████████████████████████████████████████████████| 88237/88237 [12:29<00:00, 117.75it/s]


In [11]:
len(r)

782959

In [15]:
r[-12]

{'audio_filename': 'prepared-pseudolabel-malaya-chunks/1089612-0.mp3',
 'new_text': '<|startoftranscript|><|ms|><|transcribeprecise|><|0.00|> Apa<|0.14|><|0.24|> kata<|0.42|><|0.54|> Said<|0.72|><|0.84|> Qutub<|1.10|><|1.20|> dalam<|1.40|><|1.48|> mukaddimahnya?<|2.30|><|2.80|> Al-hayatu<|3.54|><|3.68|> fi<|3.76|><|3.92|> zilalil<|4.50|><|endoftext|>'}

In [16]:
import IPython.display as ipd
ipd.Audio(r[-12]['audio_filename'])

In [17]:
with open('pseudolabel-malaya-whisper-word-timestamp.jsonl', 'w') as fopen:
    for r_ in r:
        fopen.write(f'{json.dumps(r_)}\n')
        fopen.flush()

In [18]:
!ls -lh pseudolabel-malaya-whisper-word-timestamp.jsonl

-rw-rw-r-- 1 husein husein 225M Jan   7 19:42 pseudolabel-malaya-whisper-word-timestamp.jsonl


In [19]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="pseudolabel-malaya-whisper-word-timestamp.jsonl",
    path_in_repo="pseudolabel-malaya-whisper-word-timestamp.jsonl",
    repo_id="mesolitica/Malaysian-STT-Whisper",
    repo_type="dataset",
)

pseudolabel-malaya-whisper-word-timestamp.jsonl:   0%|          | 0.00/236M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper/commit/57493cdf69fad13ea9515824246dddb9c700eef7', commit_message='Upload pseudolabel-malaya-whisper-word-timestamp.jsonl with huggingface_hub', commit_description='', oid='57493cdf69fad13ea9515824246dddb9c700eef7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/mesolitica/Malaysian-STT-Whisper', endpoint='https://huggingface.co', repo_type='dataset', repo_id='mesolitica/Malaysian-STT-Whisper'), pr_revision=None, pr_num=None)

In [20]:
!du -hs prepared-pseudolabel-malaya-chunks

14G	prepared-pseudolabel-malaya-chunks


In [22]:
!zip -q -r -s 10000m prepared-pseudolabel-malaya-chunks.zip prepared-pseudolabel-malaya-chunks

In [27]:
from huggingface_hub import HfApi
api = HfApi()

for f in glob('prepared-pseudolabel-malaya-chunks.z*'):
    print(f)
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f,
        repo_id="mesolitica/Malaysian-STT-Whisper",
        repo_type="dataset",
    )

prepared-pseudolabel-malaya-chunks.zip


prepared-pseudolabel-malaya-chunks.zip:   0%|          | 0.00/2.19G [00:00<?, ?B/s]

prepared-pseudolabel-malaya-chunks.z01


prepared-pseudolabel-malaya-chunks.z01:   0%|          | 0.00/10.5G [00:00<?, ?B/s]

In [28]:
!rm prepared-pseudolabel-malaya-chunks.z*